In [4]:
from parse import parse
A_LIST_OF_TASKS = [
    "handcrafted/c2d1_1",
    "handcrafted/mock",
    "loop_lit/afnp2014_modified",
    "loop_lit/bhmr2007",
    "loop_lit/cggmp2005",
    "loop_lit/cggmp2005_variant",
    "loop_lit/gsv2008",
    "loop_lit/gj2007",
    "loop_lit/gj2007b",
    "loop_lit/hhk2008",
    "loop_lit/jm2006",
    "loop_lit/jm2006_variant",
    "loop_new/count_by_1",
    "loop_new/count_by_1_variant",
    "loop_simple/nested_1",
    "loop_zilu/benchmark01_conjunctive",
    "loop_zilu/benchmark02_linear",
    "loop_zilu/benchmark03_linear",
    "loop_zilu/benchmark04_conjunctive",
    "loop_zilu/benchmark05_conjunctive",
    "loop_zilu/benchmark06_conjunctive",
    "loop_zilu/benchmark07_linear",
    "loop_zilu/benchmark08_conjunctive",
    "loop_zilu/benchmark09_conjunctive",
    "loop_zilu/benchmark10_conjunctive",
    "loop_zilu/benchmark11_linear",
    "loop_zilu/benchmark12_linear",
    "loop_zilu/benchmark13_conjunctive",
    "loop_zilu/benchmark14_linear",
    "loop_zilu/benchmark15_conjunctive",
    "loop_zilu/benchmark16_conjunctive",
    "loop_zilu/benchmark17_conjunctive",
    "loop_zilu/benchmark18_conjunctive",
    "loop_zilu/benchmark19_conjunctive",
    "loop_zilu/benchmark20_conjunctive",
    "loop_zilu/benchmark21_disjunctive",
    "loop_zilu/benchmark22_conjunctive",
    "loop_zilu/benchmark23_conjunctive",
    "loop_zilu/benchmark24_conjunctive",
    "loop_zilu/benchmark25_linear",
    "loop_zilu/benchmark26_linear",
    "loop_zilu/benchmark27_linear",
    "loop_zilu/benchmark28_linear",
    "loop_zilu/benchmark29_linear",
    "loop_zilu/benchmark30_conjunctive",
    "loop_zilu/benchmark31_disjunctive",
    "loop_zilu/benchmark32_linear",
    "loop_zilu/benchmark33_linear",
    "loop_zilu/benchmark34_conjunctive",
    "loop_zilu/benchmark35_linear",
    "loop_zilu/benchmark36_conjunctive",
    "loop_zilu/benchmark37_conjunctive",
    "loop_zilu/benchmark38_conjunctive",
    "loop_zilu/benchmark39_conjunctive",
    "loop_zilu/benchmark40_polynomial",
    "loop_zilu/benchmark41_conjunctive",
    "loop_zilu/benchmark42_conjunctive",
    "loop_zilu/benchmark43_conjunctive",
    "loop_zilu/benchmark44_disjunctive",
    "loop_zilu/benchmark45_disjunctive",
    "loop_zilu/benchmark46_disjunctive",
    "loop_zilu/benchmark47_linear",
    "loop_zilu/benchmark48_linear",
    "loop_zilu/benchmark49_linear",
    "loop_zilu/benchmark50_linear",
    "loop_zilu/benchmark51_polynomial",
    "loop_zilu/benchmark52_polynomial",
    "loop_zilu/benchmark53_polynomial",
    "loops_crafted_1/Mono1_1_2",
    "loops_crafted_1/Mono3_1",
]

In [6]:
# Parse log of our tool
NUM_ROUND = 3
inv_tem = 'Invariant Found:	{inv}'
mcmc_tem = "Total MCMC iterations:  {mcmc:d}"
z3_tem = "Total Z3 calls:  {z3:d}"
mcmc_time_tem = "Total MCMC time:  {mcmc_time:f}"
z3_time_tem = "Total Z3 Time:  {z3_time:f}"
thread_tem = "Number of Threads:  {thread:d}"


def read_data(file_path):
    res = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            result = parse(inv_tem, line)
            if result is not None:
                res.append({'inv': result['inv']})
                continue     
            result = parse(mcmc_tem, line)
            if result is not None:
                res[-1]['mcmc'] = result['mcmc']
                continue
            result = parse(z3_tem, line)
            if result is not None:
                res[-1]['z3'] = result['z3']
                continue
            result = parse(mcmc_time_tem, line)
            if result is not None:
                res[-1]['mcmc_time'] = result['mcmc_time']
                continue
            result = parse(z3_time_tem, line)
            if result is not None:
                res[-1]['z3_time'] = result['z3_time']
                continue
            result = parse(thread_tem, line)
            if result is not None:
                res[-1]['thread'] = result['thread']
                continue
            
    for i in range(len(res), NUM_ROUND):
        res.append(None)
    
    return res

In [5]:
import os

folder_path = 'output'
name_tem = folder_path + "/{file_name}.txt"
record = {}

def convert_filename(filename):
    tem = "{folder}.{name}"
    new_name = parse(tem, filename)
    return new_name["folder"] + "/" + new_name['name']

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        result = parse(name_tem, file_path)
        # print(result["file_name"])
        data = read_data(file_path)
        task_name = convert_filename(result["file_name"])
        record[task_name] = data
        

NameError: name 'read_data' is not defined

In [22]:
# For our tool's result formatting
def trim_dict(record):
    new_record = []
    
    for task in A_LIST_OF_TASKS:
    
        mcmc = None
        z3 = None
        if task in record:
            value = record[task]
            for i in range(NUM_ROUND):
                if value[i] is None:
                    continue
                if mcmc is None or value[i]["mcmc"] < mcmc:
                    mcmc = value[i]["mcmc"]
                    z3 = value[i]["z3"]
        if mcmc is None:
            mcmc = "-"
            z3 = "-"
         
        new_record.append((task, mcmc, z3))
    return new_record

def dict_to_latex_table(data):
    if len(data) % 2 != 0:
        missing_one = True
        length = (len(data) + 1)//2
    else:
        missing_one = False
        length = len(data)//2
    sec_col_idx = length

            
    table = "\\begin{tabular}{|l|l|l|l|l|l|}\n"
    table += "\\hline\n"
    table += "Benchmark & SA & DS & Benchmark & SA & DS \\\\\n"
    table += "\\hline\n"

    for i in range(length):
        k1 = data[i][0].replace("_", "\_")
        mcmc1 = str(data[i][1])
        z31 = str(data[i][2])
        missing_last_one = missing_one and i == length - 1
        k2 = "" if missing_last_one else data[sec_col_idx + i][0].replace("_", "\_")
        mcmc2 = "" if missing_last_one else str(data[sec_col_idx + i][1])
        z32 = "" if missing_last_one else str(data[sec_col_idx + i][2])
        table += f"{k1} & {mcmc1} & {z31} & {k2} & {mcmc2} & {z32} \\\\\n"

    table += "\\hline\n"
    table += "\\end{tabular}"

    return table

latex_table = dict_to_latex_table(trim_dict(record))
print(latex_table)

\begin{tabular}{|l|l|l|l|l|l|}
\hline
Benchmark & SA & Z3 & Benchmark & SA & Z3 \\
\hline
handcrafted/c2d1\_1 & 4176 & 3 & loop\_zilu/benchmark21\_disjunctive & 647 & 2 \\
handcrafted/mock & 287 & 4 & loop\_zilu/benchmark22\_conjunctive & - & - \\
loop\_lit/afnp2014\_modified & 1652 & 8 & loop\_zilu/benchmark23\_conjunctive & - & - \\
loop\_lit/bhmr2007 & - & - & loop\_zilu/benchmark24\_conjunctive & 83725 & 10 \\
loop\_lit/cggmp2005 & - & - & loop\_zilu/benchmark25\_linear & 167 & 4 \\
loop\_lit/cggmp2005\_variant & - & - & loop\_zilu/benchmark26\_linear & 225 & 1 \\
loop\_lit/gsv2008 & 393 & 2 & loop\_zilu/benchmark27\_linear & 1413 & 5 \\
loop\_lit/gj2007 & - & - & loop\_zilu/benchmark28\_linear & 732 & 3 \\
loop\_lit/gj2007b & - & - & loop\_zilu/benchmark29\_linear & 3404 & 13 \\
loop\_lit/hhk2008 & - & - & loop\_zilu/benchmark30\_conjunctive & 3157 & 7 \\
loop\_lit/jm2006 & - & - & loop\_zilu/benchmark31\_disjunctive & 1831 & 5 \\
loop\_lit/jm2006\_variant & - & - & loop\_zilu/ben

In [7]:
# Output the task list as a template
def output_list_of_tasks_to_file(file_name: str):
    with open(file_name, 'w') as file:
        for task in A_LIST_OF_TASKS:
            file.write(task + ",\n")

In [27]:
def read_icedt_result(path: str):
    res = []
    import os
    def file_contains_string(file_path, search_string):
        try:
            with open(file_path, "r") as file:
                for line in file:
                    if search_string in line:
                        return True
            return False
        except FileNotFoundError:
            return False
        
    for filename in A_LIST_OF_TASKS:
        filename = filename.replace("/", ".")
        filename += ".log"
        filepath = os.path.join(path, filename)
        res.append((filename, file_contains_string(filepath, "1 verified")))
    return res

def read_result(path: str):
    res = []
    with open(path, "r") as file:
        lines = file.readlines()
        for line in lines:
            ret = parse("{name},{solve}\n", line).named
            if ret["solve"] == '1':
                flag = True
            elif ret['solve'] == '0':
                flag = False
            else:
                raise Exception("Wrong flag")
            res.append((ret["name"], flag))
    return res

def comparison_result_to_latex_table(our_data, la_data, icedt_data):
    def tick_or_cross(b: bool):
        if b:
            return r"\cmark"
        else:
            return r"\xmark"
    if len(our_data) % 2 != 0:
        missing_one = True
        length = (len(our_data) + 1)//2
    else:
        missing_one = False
        length = len(our_data)//2
    sec_col_idx = length

    assert len(our_data) == len(la_data)
    assert len(our_data) == len(icedt_data)

    toolname = r"\toolname"
    la = r"\la"
    icedt = r"\icedt"
    
    table = "\\begin{tabular}{|l|l|l|l|l|l|l|l|}\n"
    table += "\\hline\n"
    table += "Benchmark & {toolname} & {la} & {icedt} & Benchmark & {toolname} & {la} & {icedt} \\\\\n"
    table += "\\hline\n"

    for i in range(length):
        k1 = our_data[i][0].replace("_", "\_")
        t1 = tick_or_cross(our_data[i][1])
        l1 = tick_or_cross(la_data[i][1])
        i1 = tick_or_cross(icedt_data[i][1])
        missing_last_one = missing_one and i == length - 1
        i = sec_col_idx + i
        k2 = "" if missing_last_one else our_data[i][0].replace("_", "\_")
        t2 = "" if missing_last_one else tick_or_cross(our_data[i][1])
        l2 = "" if missing_last_one else tick_or_cross(la_data[i][1])
        i2 = "" if missing_last_one else tick_or_cross(icedt_data[i][1])
        table += f"{k1} & {t1} & {l1} & {i1} & {k2} & {t2} & {l2} & {i2} \\\\\n"

    table += "\\hline\n"
    table += "\\end{tabular}"

    return table

In [19]:
our_data = read_icedt_result("../bench/result/icedt_log")
# la_data = read_icedt_result("../bench/result/icedt_log")
# icedt_data = read_icedt_result("../bench/result/icedt_log")


NameError: name 'data' is not defined

In [28]:

print(comparison_result_to_latex_table(our_data, our_data, our_data))


\begin{tabular}{|l|l|l|l|l|l|l|l|}
\hline
Benchmark & {toolname} & {la} & {icedt} & Benchmark & {toolname} & {la} & {icedt} \\
\hline
handcrafted.c2d1\_1.log & \cmark & \cmark & \cmark & loop\_zilu.benchmark21\_disjunctive.log & \xmark & \xmark & \xmark \\
handcrafted.mock.log & \cmark & \cmark & \cmark & loop\_zilu.benchmark22\_conjunctive.log & \cmark & \cmark & \cmark \\
loop\_lit.afnp2014\_modified.log & \xmark & \xmark & \xmark & loop\_zilu.benchmark23\_conjunctive.log & \xmark & \xmark & \xmark \\
loop\_lit.bhmr2007.log & \xmark & \xmark & \xmark & loop\_zilu.benchmark24\_conjunctive.log & \xmark & \xmark & \xmark \\
loop\_lit.cggmp2005.log & \cmark & \cmark & \cmark & loop\_zilu.benchmark25\_linear.log & \cmark & \cmark & \cmark \\
loop\_lit.cggmp2005\_variant.log & \xmark & \xmark & \xmark & loop\_zilu.benchmark26\_linear.log & \cmark & \cmark & \cmark \\
loop\_lit.gsv2008.log & \cmark & \cmark & \cmark & loop\_zilu.benchmark27\_linear.log & \xmark & \xmark & \xmark \\
loop\_li